In [1]:
import tensorflow as tf
import pickle
import numpy as np
import timeit
import load_cifer
from scipy.sparse import csr_matrix

In [2]:
tf.reset_default_graph()

<h1>Define Hyper-perparmeter</h1>

In [3]:
epochs = 10
batch_size = 128
keep_probability = 0.7
learning_rate = 0.001

<h1>Define Placeholder</h1>

In [4]:
# Inputs
x = tf.placeholder(tf.float32, shape=(None, 32, 32, 3), name='input_x')
y =  tf.placeholder(tf.float32, shape=(None, 10), name='output_y')
keep_prob = tf.placeholder(tf.float32, name='keep_prob')

<h1>Define Neural Network Architecture</h1>

In [5]:
def fullconn_net(x, keep_prob):
    #x=tf.get_variable('x',shape=[x.shape[0],256],initializer=tf.contrib.layers.variance_scaling_initializer())
    flat = tf.contrib.layers.flatten(x)
    full1 = tf.contrib.layers.fully_connected(inputs=flat, num_outputs=2048, activation_fn=tf.nn.elu, weights_initializer=tf.contrib.layers.variance_scaling_initializer())
    full1 = tf.nn.dropout(full1, keep_prob)
    full1 = tf.layers.batch_normalization(full1)
    
    full2 = tf.contrib.layers.fully_connected(inputs=full1, num_outputs=1024, activation_fn=tf.nn.elu, weights_initializer=tf.contrib.layers.variance_scaling_initializer())
    full2 = tf.nn.dropout(full2, keep_prob)
    full2 = tf.layers.batch_normalization(full2)   
    
    full3 = tf.contrib.layers.fully_connected(inputs=full2, num_outputs=512, activation_fn=tf.nn.elu, weights_initializer=tf.contrib.layers.variance_scaling_initializer())
    full3 = tf.nn.dropout(full3, keep_prob)
    full3 = tf.layers.batch_normalization(full3)   
    
    out = tf.contrib.layers.fully_connected(inputs=full3, num_outputs=10, activation_fn=None)
    return out

<h1>Define cost andoptimization</h1>

In [6]:
logits = fullconn_net(x, keep_prob)

# Loss and Optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



<h1>Training and testing</h1>
<h2>1.Print out validation accuracy after each training poch</h2>
<h2>2.Print out training time you spend on each epoch</h2>
<h2>3.Print out testing accuracy in the end</h2>

In [7]:
def train_neural_network(session, optimizer, feature_batch, label_batch, keep_probability):
    session.run(optimizer, 
                feed_dict={
                    x: feature_batch,
                    y: label_batch,
                    keep_prob: keep_probability
                })

In [8]:
def print_stats(session, feature_batch, label_batch, valid_features, valid_labels, cost, accuracy):
    loss = sess.run(cost, 
                    feed_dict={
                        x: feature_batch,
                        y: label_batch,
                        keep_prob: 1
                    })
    valid_acc = sess.run(accuracy, 
                         feed_dict={
                             x: valid_features,
                             y: valid_labels,
                             keep_prob: 1
                         })
    
    print('Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(loss, valid_acc))

In [9]:
features_train=pickle.load(open('features_train.pkl','rb'))
features_valid=pickle.load(open('features_valid.pkl','rb'))
features_test=pickle.load(open('features_test.pkl','rb'))
labels_train=pickle.load(open('labels_train.pkl','rb'))
labels_valid=pickle.load(open('labels_valid.pkl','rb'))
labels_test=pickle.load(open('labels_test.pkl','rb'))

In [10]:
sess=tf.Session()
# Initializing the variables
sess.run(tf.global_variables_initializer())

# Training cycle
for epoch in range(epochs):
    # Loop over all batches
    n_batches = 5
    for i in range(n_batches):
        train_neural_network(sess, optimizer, features_train[i], labels_train[i].toarray(), keep_probability)

        print('Epoch {:>2}, CIFAR-10 Batch {}:  '.format(epoch + 1, i+1), end='')
        print_stats(sess, features_train[i], labels_train[i].toarray(), features_valid[i], labels_valid[i].toarray(), cost, accuracy)

Epoch  1, CIFAR-10 Batch 1:  Loss:     2.4039 Validation Accuracy: 0.140125
Epoch  1, CIFAR-10 Batch 2:  Loss:     2.4774 Validation Accuracy: 0.132375
Epoch  1, CIFAR-10 Batch 3:  Loss:     2.5388 Validation Accuracy: 0.123875
Epoch  1, CIFAR-10 Batch 4:  Loss:     2.4943 Validation Accuracy: 0.126125
Epoch  1, CIFAR-10 Batch 5:  Loss:     2.3789 Validation Accuracy: 0.138250
Epoch  2, CIFAR-10 Batch 1:  Loss:     2.1570 Validation Accuracy: 0.139625
Epoch  2, CIFAR-10 Batch 2:  Loss:     2.1838 Validation Accuracy: 0.148625
Epoch  2, CIFAR-10 Batch 3:  Loss:     2.2677 Validation Accuracy: 0.131625
Epoch  2, CIFAR-10 Batch 4:  Loss:     2.2119 Validation Accuracy: 0.129875
Epoch  2, CIFAR-10 Batch 5:  Loss:     2.2281 Validation Accuracy: 0.135125
Epoch  3, CIFAR-10 Batch 1:  Loss:     2.0420 Validation Accuracy: 0.144750
Epoch  3, CIFAR-10 Batch 2:  Loss:     2.0743 Validation Accuracy: 0.151375
Epoch  3, CIFAR-10 Batch 3:  Loss:     2.1122 Validation Accuracy: 0.148000
Epoch  3, CI

In [11]:
result=sess.run(accuracy,feed_dict={x: features_test,y: labels_test.toarray(),keep_prob: 1})

In [12]:
print(result)

0.1439
